# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [19]:
# import dependencies
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
#import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [20]:
# initialise workspace
ws = Workspace.from_config() # using from_config file downloaded from Azure ML Studio workspace. requires authentication

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


Workspace name: quick-starts-ws-134289
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-134289


In [21]:
# create an AutoML experiment

# experiment name
experiment_name = 'ctg_experiment'
project_folder = './ctgautomlproject'

# create the experiment
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ctg_experiment,quick-starts-ws-134289,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
# create or attach an existing compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# set name of CPU cluster
cpu_cluster_name = "ctg-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, using it.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# wait for completion
cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, using it.

Running


## Dataset

### Overview

This project uses the cardiotocogram data set available at the UCI ML website.
The dataset consists of measurements of fetal heart rate (FHR) and uterine contraction (UC) features on cardiotocograms classified by expert obstetricians. It consists of 2126 fetal cardiotocograms (CTGs) with 23 attributes.

The CTGs were automatically processed and the respective diagnostic features measured and classified by three expert obstetricians and a consensus classification label assigned to each of them. Classification was both with respect to a morphologic pattern (A, B, C, ..., J) and to a fetal state (N, S, P). Therefore the dataset can be used either for 10-class or 3-class experiments. 

In this work, the 3-class label is used to classify a foetus as Normal, Suspect or Pathological based on the cardiotocogram measurements.



In [23]:
# import data external to AzureML
from azureml.data.dataset_factory import TabularDatasetFactory

# first try to load the dataset from the workspace. Otherwise, create it from the file

found = False
key = "ctg-data"
description_text = "Cardiocotogram dataset for fetal state classification Udacity Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        print('found dataset in workspace')
        ds = ws.datasets[key] 

if not found:
    # Create TabularDataset using TabularDatasetFactory
    url = "https://raw.githubusercontent.com/ijeendu/Machine_Learning_in_Microsoft_Azure/main/CTG.csv"
    ds = TabularDatasetFactory.from_delimited_files(path=url)
    
    # register dataset in workspace
    ds = ds.register(workspace=ws,
                    name=key,
                    description=description_text)

# view dataset
ds.to_pandas_dataframe().head()

found dataset in workspace


,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,Variab10.txt,1996-12-01,CTG0001.txt,240,357,120,120,0,0,0,...,0,0,0,0,0,0,1,0,9,2
1,Fmcs_1.txt,1996-05-03,CTG0002.txt,5,632,132,132,4,0,4,...,0,0,0,1,0,0,0,0,6,1
2,Fmcs_1.txt,1996-05-03,CTG0003.txt,177,779,133,133,2,0,5,...,0,0,0,1,0,0,0,0,6,1
3,Fmcs_1.txt,1996-05-03,CTG0004.txt,411,1192,134,134,2,0,6,...,0,0,0,1,0,0,0,0,6,1
4,Fmcs_1.txt,1996-05-03,CTG0005.txt,533,1147,132,132,4,0,5,...,0,0,0,0,0,0,0,0,2,1


In [24]:
# explore dataframe
df = ds.to_pandas_dataframe()
df.describe()

,b,e,LBE,LB,AC,FM,UC,ASTV,MSTV,ALTV,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
count,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00,...,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00,2126.00
mean,878.44,1702.88,133.30,133.30,2.72,7.24,3.66,46.99,1.33,9.85,...,0.02,0.04,0.03,0.16,0.12,0.05,0.03,0.09,4.51,1.30
std,894.08,930.92,9.84,9.84,3.56,37.13,2.85,17.19,0.88,18.40,...,0.16,0.19,0.18,0.36,0.32,0.22,0.18,0.29,3.03,0.61
min,0.00,287.00,106.00,106.00,0.00,0.00,0.00,12.00,0.20,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00
25%,55.00,1009.00,126.00,126.00,0.00,0.00,1.00,32.00,0.70,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,1.00
50%,538.00,1241.00,133.00,133.00,1.00,0.00,3.00,49.00,1.20,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,1.00
75%,1521.00,2434.75,140.00,140.00,4.00,2.00,5.00,61.00,1.70,11.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.00,1.00
max,3296.00,3599.00,160.00,160.00,26.00,564.00,23.00,87.00,7.00,91.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,10.00,3.00


In [25]:
# prepare dataset for model training

from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Clean dataset using the clean_data function
x, y = clean_data(ds)

# split dataset into train and test sets
(x_train, x_test, y_train, y_test) = train_test_split(x, y, test_size= 0.3, random_state = 0)
label = 'NSP'

# merge the output x and y dataframes into a single table for AutoML experiment
train_data_df = pd.concat([x_train, y_train], axis=1)

# view train dataset
train_data_df.head()

,b,e,LBE,LB,AC,FM,UC,ASTV,MSTV,ALTV,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
994,44,831,129,129,3,0,1,27,1.20,1,...,0,0,0,0,0,0,0,0,2,1
1618,848,2108,142,142,4,63,3,66,3.30,0,...,0,0,0,1,0,0,0,0,6,1
177,0,1033,119,119,8,0,5,30,1.80,0,...,0,0,0,0,0,0,0,0,2,1
519,25,679,126,126,5,26,3,42,1.50,0,...,0,0,0,0,0,0,0,0,2,1
745,242,1014,142,142,1,0,4,23,3.60,0,...,0,0,0,0,1,0,0,0,7,1


In [26]:
# save training data in tabular format to allow for remote run
if not os.path.isdir('data'):  # create data folder if it does not exist
    os.mkdir('data')
    
# Save the train data to a csv file to be uploaded to the datastore
pd.DataFrame(train_data_df).to_csv("data/train_data.csv", index=False)

# Upload the training data as a tabular dataset for access during training on remote compute
# upload to data store
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='ctgclassifier', overwrite=True, show_progress=True)

 # access datastore during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('ctgclassifier/train_data.csv'))


Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [27]:
# Set parameters for AutoMLConfig

# define autoconfig settings
automl_settings = {
    'enable_early_stopping' : True,
    'iteration_timeout_minutes': 5,
    'max_concurrent_iterations': 4,
    'max_cores_per_iteration': -1,    
    'featurization': 'auto',
    'verbosity': logging.INFO,
}

# define automl autconfig parameters
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric='accuracy',
    enable_onnx_compatible_models=True,
    compute_target=cpu_cluster, 
    training_data= train_data,
    label_column_name= label,
    path = project_folder,
    n_cross_validations=3,
    debug_log = 'automl_errors.log',
    **automl_settings)

In [28]:
#Submit experiment for remote run
automl_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on ctg-cluster with default configuration
Running on remote compute: ctg-cluster
Parent Run ID: AutoML_5fcf73f9-b3a9-4471-8dab-d7be02ded432

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
# show run details
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

# wait for completion
automl_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_5fcf73f9-b3a9-4471-8dab-d7be02ded432',
 'target': 'ctg-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-11T04:07:46.892771Z',
 'endTimeUtc': '2021-01-11T04:26:03.502373Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'ctg-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"ctg_experiment","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-134289","workspace_name":"quick-starts-ws-134289","region":"southcentralus","compute_target":"ctg-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":null,"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [30]:
# Retrieve the best automl run model
best_automl_run, fitted_automl_model = automl_run.get_output()
print('Best AutoML run: ', best_automl_run)
print('Best AutoML model :', fitted_automl_model)

# get best model and display properties
model_name = best_automl_run.properties['model_name']
print('Best_model name: ', model_name)

# display all the properties of the best model
best_automl_run.get_properties()

Best AutoML run:  Run(Experiment: ctg_experiment,
Id: AutoML_5fcf73f9-b3a9-4471-8dab-d7be02ded432_46,
Type: azureml.scriptrun,
Status: Completed)
Best AutoML model : Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                num_leaves=2,
                                                                                                objective=None,
    

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'ctg_experiment\',\'compute_target\':\'ctg-cluster\',\'subscription_id\':\'510b94ba-e453-4417-988b-fbdc37b55ca7\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_5fcf73f9-b3a9-4471-8dab-d7be02ded432_46","experiment_name":"ctg_experiment","workspace_name":"quick-starts-ws-134289","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group_name":"aml-quickstarts-134289"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '46',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.ComputeTargetTy

In [31]:
#TODO: Save the best AutoML model
import joblib

joblib.dump(fitted_automl_model, 'best_automl_model.pkl')

['best_automl_model.pkl']

In [32]:
# register best AutoML model for future deployment
from azureml.core.model import Model
description = 'AutoML Model trained on cardiocotogram data to predict fetal state as either Normal, Suspect or Pathologic'
tags = {'area': 'neonatal health', 'type': 'classification'}

automl_model = Model.register(workspace =ws, model_name = 'best-ctgAutoMLmodel', model_path = 'best_automl_model.pkl', description = description, tags = tags)

print('AutoML RunID: ', automl_run.id, sep='\t')

Registering model best-ctgAutoMLmodel
AutoML RunID: 	AutoML_5fcf73f9-b3a9-4471-8dab-d7be02ded432


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [33]:
# deploy model as a web service
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies

service_name = 'ctg-classification-service'

env = Environment.get(workspace=ws, name='AzureML-Tutorial')
conda_dep = CondaDependencies()

# Install scikit-learn version 0.22 and xgbosst version 0.22 conda packages
conda_dep.add_conda_package('scikit-learn==0.22')
conda_dep.add_conda_package('xgboost==0.90')

# Add the dependencies to PythonSection of my conda env
env.python.conda_dependencies=conda_dep


#env.python.conda_dependencies.add_pip_package("scikit-learn==0.22", 'xgboost==0.90')
#env = Environment.from_conda_specification(name='project-env', file_path='myenv.yml')

#for pip_package in ["scikit-learn"]:
    #env.python.conda_dependencies.add_pip_package(pip_package)

#inference_config = InferenceConfig(runtime='python', entry_script='score.py', conda_file=myenv)
inference_config = InferenceConfig(entry_script='score.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True, auth_enabled=True)

model = Model(ws,name='best-ctgAutoMLmodel')
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [34]:
print(service.get_logs())


/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-11T04:28:14,340790427+00:00 - iot-server/run 
2021-01-11T04:28:14,340030421+00:00 - gunicorn/run 
2021-01-11T04:28:14,340869727+00:00 - nginx/run 
2021-01-11T04:28:14,348591982+00:00 - rsyslog/run 
rsyslogd

In [35]:
# list the packages contained in the AzureML-Tutorial curated environment

envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith('AzureML'):
        print('Name',env)
        print('packages', envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-VowpalWabbit-8.8.0
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.19.0
  - azureml-defaults==1.19.0
  - azureml-dataset-runtime[fuse,pandas]
name: azureml_769be4b756b756954fa484d1287d5153

Name AzureML-AutoML
packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.19.0
  - azureml-pipeline-core==1.19.0
  - azureml-telemetry==1.19.0
  - azureml-defaults==1.19.0
  - azureml-interpret==1.19.0
  - azureml-automl-core==1.19.0
  - azureml-automl-runtime==1.19.0
  - azureml-train-automl-client==1.19.0
  - azureml-train-automl-runtime==1.19.0
  - azureml-dataset-runtime==1.19.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - boto3==1.15.18
  - botocore==1.18.18
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_8eff28b157f42edcd2424a5aae6c8074

Name AzureML-Hyperd

In [36]:
print(service.state)
print('scoring URI: ' + service.scoring_uri)

print('Swagger URI: ' + service.swagger_uri)

primary, secondary = service.get_keys()

print('Primary Authentication Key: ' + primary)

Healthy
scoring URI: http://825d591f-68e0-4ed6-b214-5f68b420bf1b.southcentralus.azurecontainer.io/score
Swagger URI: http://825d591f-68e0-4ed6-b214-5f68b420bf1b.southcentralus.azurecontainer.io/swagger.json
Primary Authentication Key: S8P6zihYx1UfkUa06LaKtlL7VCoOseOE


In [103]:
import requests 
import json  
scoring_uri = 'http://825d591f-68e0-4ed6-b214-5f68b420bf1b.southcentralus.azurecontainer.io/score' # update with info from previous cell
key = 'S8P6zihYx1UfkUa06LaKtlL7VCoOseOE'            # update with info from previous cell  

data = {"data": 
            [
                {
                    'b': 0.00,
                    'e': 1199.00,
                    'LBE': 151.00,
                    'LB': 151.00,
                    'AC': 0.00,
                    'FM': 0.00,
                    'UC': 1.00,
                    'ASTV': 64.00,
                    'MSTV': 1.90,
                    'ALTV': 9.00,
                    'MLTV': 27.60,
                    'DL': 1.00,
                    'DS': 0.00,
                    'DP': 0.00,
                    'DR': 0.00,
                    'Width': 130.00,
                    'Min': 56.00,
                    'Max': 186.00,
                    'Nmax': 2.00,
                    'Nzeros': 0.00,
                    'Mode': 150.00,
                    'Mean': 148.00,
                    'Median': 151.00,
                    'Variance': 9.00,
                    'Tendency': 1.00,
                    'A': 0.00,
                    'B': 0.00,
                    'C': 0.00,
                    'D': 0.00,
                    'E': 0.00,
                    'AD': 0.00,
                    'DE': 0.00,
                    'LD': 0.00,
                    'FS': 0.00,
                    'SUSP': 1.00,
                    'CLASS': 10.00
                },
                {
                    'b': 0.00,
                    'e': 718.00,
                    'LBE': 130.0,
                    'LB': 130.00,
                    'AC': 4.00,
                    'FM': 273.00,
                    'UC': 3.00,
                    'ASTV': 19.00,
                    'MSTV': 2.30,
                    'ALTV': 0.00,
                    'MLTV': 8.70,
                    'DL': 3.00,
                    'DS': 0.00,
                    'DP': 1.00,
                    'DR': 0.00,
                    'Width': 107.00,
                    'Min': 67.00,
                    'Max': 174.00,
                    'Nmax': 3.00,
                    'Nzeros': 0.00,
                    'Mode': 134.00,
                    'Mean': 127.00,
                    'Median': 133.00,
                    'Variance': 43.00,
                    'Tendency': 0.00,
                    'A': 0.00,
                    'B': 0.00,
                    'C': 0.00,
                    'D': 0.00,
                    'E': 0.00,
                    'AD': 1.00,
                    'DE': 0.00,
                    'LD': 0.00,
                    'FS': 0.00,
                    'SUSP': 0.00,
                    'CLASS': 6.00
                }    
            ]
        } 

#print(data)

input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'} 
headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoring_uri, input_data, headers=headers) 
print(response.text)
print(response.status_code)

"DataErrorException:\n\tMessage: df should be a pandas dataframe or numpy array\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"code\": \"System\",\n        \"inner_error\": {\n            \"code\": \"DataError\"\n        },\n        \"message\": \"df should be a pandas dataframe or numpy array\",\n        \"target\": \"utilities.check_input\",\n        \"reference_code\": \"098d83d9-f525-4844-9666-ea28b1a41f08\"\n    }\n}"
200


TODO: In the cell below, send a request to the web service you deployed to test it.

In [83]:
x1 = x_test.iloc[10, :-1].tolist()
print(x1)
x2 = x_test.iloc[15, :-1].tolist()
print(x2)
y1 = y_test.iloc[10]
print(y1)
y2 = y_test.iloc[15]
print(y2)

[136.0, 454.0, 130.0, 130.0, 0.0, 110.0, 0.0, 80.0, 0.3, 0.0, 7.4, 0.0, 0.0, 1.0, 0.0, 49.0, 96.0, 145.0, 3.0, 0.0, 108.0, 123.0, 122.0, 39.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[535.0, 1201.0, 122.0, 122.0, 2.0, 4.0, 4.0, 22.0, 7.0, 0.0, 16.6, 4.0, 0.0, 0.0, 0.0, 149.0, 50.0, 199.0, 7.0, 0.0, 129.0, 118.0, 124.0, 75.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
3
1


In [94]:
import requests 
import json  
scoring_uri = 'http://825d591f-68e0-4ed6-b214-5f68b420bf1b.southcentralus.azurecontainer.io/score' # update with info from previous cell
key = 'S8P6zihYx1UfkUa06LaKtlL7VCoOseOE'            # update with info from previous cell  

data = {"data": [x1, x2]
                        
       } 

#print(data)

input_data = json.dumps(data)
#input_data = json.dumps(np.array([x1, x2]).tolist())
headers = {'Content-Type': 'application/json'} 
headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoring_uri, input_data, headers=headers) 
print(response.text)

"DataErrorException:\n\tMessage: df should be a pandas dataframe or numpy array\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"code\": \"System\",\n        \"inner_error\": {\n            \"code\": \"DataError\"\n        },\n        \"message\": \"df should be a pandas dataframe or numpy array\",\n        \"target\": \"utilities.check_input\",\n        \"reference_code\": \"098d83d9-f525-4844-9666-ea28b1a41f08\"\n    }\n}"


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# print logs
from azureml.core import Workspace
from azureml.core.webservice import Webservice

# Requires the config to be downloaded first to the current working directory
ws = Workspace.from_config()

# Set with the deployment name
name = "ctg-classification-service"
# load existing web service
service = Webservice(name=name, workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)


In [ ]:
#delete service
service.delete()